In [52]:
from deta import Deta
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
from yahoo_fin.stock_info import *
import plotly.graph_objects as go

In [13]:
class StockyDb:
    def __init__(self):
        self.deta = Deta('d0p5if1f_GSnmoPk32YPhwKaJzN6sq7hM2DN4XPks')
        self.db = self.deta.Base('StockyAI')
        
    def save_to_DB(self,ticker,df):
        date_=[]
        open_=[]
        high_=[]
        low_=[]
        close_=[]
        for i in range (0,len(df)):
            df['Close']=df['Close'].astype(float)
            df['Low']=df['Low'].astype(float)
            df['High']=df['High'].astype(float)
            df['Open']=df['Open'].astype(float)
            date_.append(df['Date'][i])
            open_.append(df['Open'][i])
            high_.append(df['High'][i])
            low_.append(df['Low'][i])
            close_.append(df['Close'][i])
        self.db.put({'key':ticker,'date':date_,'open':open_,'high':high_,'low':low_,'close':close_})
    
    def ticker_df(self,ticker):
        return self.db.get(key=ticker)
    
    def get_all_ticker(self):
        tickers=[]
        for i in range(0,self.db.fetch().count):
            tickers.append(self.db.fetch().items[i]['key'])
        return tickers
    
    def Recomodation(self):
    tickers = DB.get_all_ticker()
    buy=[]
    sell = []
    
    for i in tickers:
        rec={}
        df=pd.DataFrame(DB.ticker_df(i))
        H_max=df['high'].max()
        L_min=df['low'].min()
        H_date=(df[df['high']==H_max])['date'].to_list()
        L_date=(df[df['low']==L_min])['date'].to_list()
        if H_date>L_date:
            price={}
            price['Ticker']=i
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            buy.append(price)
        elif H_date<L_date:
            price={}
            price['Ticker']=i
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            sell.append(price)
    
    buy_df=pd.DataFrame()
    for i in range (0,len(buy)):
        buy_df=buy_df.append([buy[i]],ignore_index=True)
    sell_df=pd.DataFrame()
    for j in range (0,len(sell)):
        sell_df=sell_df.append([sell[j]],ignore_index=True)
    return buy_df,sell_df

In [161]:
dbp = deta.Base('StockyAI_portfolio')

In [392]:
class Portfolio():
    def __init__(self):
        self.dbp = deta.Base('StockyAI_portfolio')
        self.p_data = self.dbp.get(key='praveen')
        self.cash = self.p_data['cash']
        self.stocks = self.p_data['stocks']
        #self.S_df = pd.DataFrame(columns=['stock','buy_price','quantity'])
        #self.S_df=self.S_df.append(self.stocks,ignore_index=True)
        print(self.stocks)
        
    def Buy (self,ticker):
        tik= ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                S_detail={}
                S_detail['buy_price']=c_price
                S_detail['quantity'] = quant
                self.stocks[tik]=S_detail
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                print('stocks brought')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')

    def Sell (self,ticker):
        tik=ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.stocks[tik]['quantity']>= quant :
                self.cash=self.cash+amt
                r_quant = self.stocks[tik]['quantity']-quant
                S_details=(self.stocks[tik])
                S_details['quantity']=r_quant
                self.stocks[tik]=S_details
                #print(self.stocks[tik])
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if cash >= 0:
                cash=cash+amt
                print('stocks Sold')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')
        
    def get_holdings(self):
        #print(self.p_data['stocks'])
        hold_df = pd.DataFrame(self.p_data['stocks'])
        hold_df=hold_df.transpose()
        hold_df=hold_df.reset_index()
        hold_df['index']=hold_df['index'].astype('str')
        tiks=hold_df['index'].to_list()
        #tickers=yf.Tickers(tiks)
        prices=[]
        
        for i in (tiks):
            #i = tickers.tickers[i]
            pr = get_live_price(i)
            #pr = pr['regularMarketPrice']
            prices.append(pr)
            
        #hold_df['current price']=hold_df['index'].apply(self.get_current_price)
        hold_df['current price']=prices
        return hold_df
    
    def get_current_price(self,ticker):
        ticker=yf.Ticker(ticker)
        info=ticker.info
        return info['regularMarketPrice']

In [3]:
deta = Deta('d0p5if1f_GSnmoPk32YPhwKaJzN6sq7hM2DN4XPks')
dbh = deta.Base('StockyAI_home')

In [6]:
closing_price=dbh.get(key='closing price')
closing_price=closing_price['price']

In [7]:
Liveprices=dbh.get(key='Live prices')
Live_prices=Live_prices['price']

In [21]:
Live_prices['APOLLOHOSP.NS']

4700.14990234375

In [22]:
N50List= tickers_nifty50()

In [23]:
N50List

['ADANIENT.NS',
 'APOLLOHOSP.NS',
 'BAJAJ-AUTO.NS',
 'BAJAJFINSV.NS',
 'BAJFINANCE.NS',
 'BHARTIARTL.NS',
 'BRITANNIA.NS',
 'CIPLA.NS',
 'COALINDIA.NS',
 'HDFCLIFE.NS',
 'HEROMOTOCO.NS',
 'HINDALCO.NS',
 'ICICIBANK.NS',
 'INDUSINDBK.NS',
 'ITC.NS',
 'KOTAKBANK.NS',
 'LT.NS',
 'MARUTI.NS',
 'MM.NS',
 'NESTLEIND.NS',
 'NTPC.NS',
 'ONGC.NS',
 'RELIANCE.NS',
 'TATACONSUM.NS',
 'TATASTEEL.NS',
 'TCS.NS',
 'TECHM.NS',
 'TITAN.NS',
 'ULTRACEMCO.NS',
 'WIPRO.NS']

In [19]:
change_price={}
for i in (N50List):
    change_price[i]=Live_prices[i]-closing_price[i]

In [24]:
change_price['ADANIENT.NS']

0.0

In [36]:
def N50_change():
        N50List = tickers_nifty50()
        closing_price=dbh.get(key='closing price')
        closing_price=closing_price['price']
        Live_prices=dbh.get(key='Live prices')
        Live_pri=Live_prices['price']
        change_price={}
        for i in (N50List):
            change_price[i]=Live_pri[i]-closing_price[i]

        return change_price,N50List,Live_pri

In [37]:
change_price,N50_list,N50Live=N50_change()

In [38]:
N50Live

{'ADANIENT.NS': 3642.199951171875,
 'APOLLOHOSP.NS': 4700.14990234375,
 'BAJAJ-AUTO.NS': 3541.800048828125,
 'BAJAJFINSV.NS': 1497.050048828125,
 'BAJFINANCE.NS': 6374.60009765625,
 'BHARTIARTL.NS': 808.6500244140625,
 'BRITANNIA.NS': 4331.89990234375,
 'CIPLA.NS': 1119.1500244140625,
 'COALINDIA.NS': 215.0500030517578,
 'HDFCLIFE.NS': 565.4500122070312,
 'HEROMOTOCO.NS': 2636.85009765625,
 'HINDALCO.NS': 429.54998779296875,
 'ICICIBANK.NS': 878.9000244140625,
 'INDUSINDBK.NS': 1147.8499755859375,
 'ITC.NS': 326.70001220703125,
 'KOTAKBANK.NS': 1821.949951171875,
 'LT.NS': 2062.75,
 'MARUTI.NS': 8141.60009765625,
 'MM.NS': 1224.5999755859375,
 'NESTLEIND.NS': 20132.75,
 'NTPC.NS': 163,
 'ONGC.NS': 139.8000030517578,
 'RELIANCE.NS': 2502.199951171875,
 'TATACONSUM.NS': 779,
 'TATASTEEL.NS': 102.25,
 'TCS.NS': 3228.35009765625,
 'TECHM.NS': 995.75,
 'TITAN.NS': 2483.050048828125,
 'ULTRACEMCO.NS': 6909.39990234375,
 'WIPRO.NS': 377.6499938964844}

In [70]:
def stock1Mp(ticker):
    tick =yf.Ticker(ticker)
    his = tick.history(period='1mo')
    his=his.reset_index()
    his.drop((['Dividends','Stock Splits']),axis=1,inplace=True)
    return his

In [71]:
his=stock1Mp('TATAMOTORS.NS')

In [72]:
his

,Date,Open,High,Low,Close,Volume
0,2022-11-23,426.850006,429.299988,422.850006,424.049988,6568241
1,2022-11-24,424.500000,425.250000,420.700012,423.399994,6768613
2,2022-11-25,425.000000,435.149994,424.000000,433.200012,18764073
3,2022-11-28,435.000000,440.350006,432.399994,433.250000,10701208
4,2022-11-29,432.500000,437.899994,429.549988,433.149994,8630105
5,2022-11-30,435.000000,441.000000,433.299988,439.399994,14727892
6,2022-12-01,443.000000,443.600006,436.850006,438.250000,8909310
7,2022-12-02,439.000000,441.149994,435.000000,435.399994,8352658
8,2022-12-05,435.500000,437.500000,427.799988,428.750000,12857807
9,2022-12-06,428.799988,431.350006,425.350006,427.100006,8461831


In [53]:
go.Line(his['Close'])

{Timestamp('2022-11-23 00:00:00'): 424.04998779296875,
 Timestamp('2022-11-24 00:00:00'): 423.3999938964844,
 Timestamp('2022-11-25 00:00:00'): 433.20001220703125,
 Timestamp('2022-11-28 00:00:00'): 433.25,
 Timestamp('2022-11-29 00:00:00'): 433.1499938964844,
 Timestamp('2022-11-30 00:00:00'): 439.3999938964844,
 Timestamp('2022-12-01 00:00:00'): 438.25,
 Timestamp('2022-12-02 00:00:00'): 435.3999938964844,
 Timestamp('2022-12-05 00:00:00'): 428.75,
 Timestamp('2022-12-06 00:00:00'): 427.1000061035156,
 Timestamp('2022-12-07 00:00:00'): 420.1000061035156,
 Timestamp('2022-12-08 00:00:00'): 417.20001220703125,
 Timestamp('2022-12-09 00:00:00'): 413.29998779296875,
 Timestamp('2022-12-12 00:00:00'): 414.1499938964844,
 Timestamp('2022-12-13 00:00:00'): 418.5,
 Timestamp('2022-12-14 00:00:00'): 417.8999938964844,
 Timestamp('2022-12-15 00:00:00'): 416.70001220703125,
 Timestamp('2022-12-16 00:00:00'): 421.6000061035156,
 Timestamp('2022-12-19 00:00:00'): 418.0,
 Timestamp('2022-12-20 00:

In [74]:
int(get_live_price('^NSEI'))

17806